## Start Node

In [1]:
import rclpy
from nimbro_utils.lazy import start_and_spin_node, stop_node

In [2]:
class MyNode(rclpy.node.Node):
    def __init__(self, context=None):
        super().__init__("test_logger_node", context=context)

In [3]:
node, executor, context, thread = start_and_spin_node(MyNode, blocking=False)

> Starting node 'MyNode'


## Replace default Logger

In [4]:
from nimbro_utils.lazy import Logger

In [5]:
node._logger = Logger(node)

#### The Logger can now be accessed directly or via the the Node's `get_logger()` function.

In [6]:
node._logger == node.get_logger()

True

In [7]:
node._logger.info("Hello World!")
node.get_logger().info("Hello World!")
None # prevents printing return value

2025-09-10 18:42:19.888 [INFO] [test_logger_node]: Hello World!
2025-09-10 18:42:19.890 [INFO] [test_logger_node]: Hello World!


#### The underlying RcutilsLogger can be accessed by using the Logger's `get_logger()` function.

In [8]:
print("Type of 'node.get_logger()':", type(node.get_logger()))
print("Type of 'node._logger.get_logger()':", type(node._logger.get_logger()))

Type of 'node.get_logger()': <class 'nimbro_utils.node_extensions.logger.Logger'>
Type of 'node._logger.get_logger()': <class 'rclpy.impl.rcutils_logger.RcutilsLogger'>


#### A Logger provides the same logging functions and keyword arguments as RcutilsLogger.

In [9]:
node._logger.debug("Testing debug()") # not logged because default severity is 20
node._logger.info("Testing info()")
node._logger.warn("Testing warn()")
node._logger.error("Testing error()")
node._logger.fatal("Testing fatal()")
None

2025-09-10 18:42:19.907 [INFO] [test_logger_node]: Testing info()
2025-09-10 18:42:19.910 [WARN] [test_logger_node]: Testing warn()
2025-09-10 18:42:19.912 [ERROR] [test_logger_node]: Testing error()
2025-09-10 18:42:19.914 [FATAL] [test_logger_node]: Testing fatal()


In [10]:
node._logger.info("Testing once", once=True)
node._logger.info("Testing skip_first", skip_first=True)
node._logger.info("Testing throttle_duration_sec", throttle_duration_sec=5.0)
None

2025-09-10 18:42:19.925 [INFO] [test_logger_node]: Testing once
2025-09-10 18:42:19.929 [INFO] [test_logger_node]: Testing throttle_duration_sec


#### The Logger's settings can be obtained via `get_settings()` and updated via `set_settings()`.

In [11]:
node._logger.debug("Testing debug()") # returns False because it is not logged

False

In [12]:
node._logger.get_settings()

{'severity': 20, 'prefix': None, 'name': None, 'settings_debug': False}

In [13]:
node._logger.set_settings({'severity': 10})

In [14]:
node._logger.debug("Testing debug()")

2025-09-10 18:42:19.966 [DEBUG] [test_logger_node]: Testing debug()


True

## Add additional Loggers

In [15]:
node.second_logger = Logger(node, settings={'name': "test"})

In [16]:
node.second_logger.info("Hello from another Logger!")
None

2025-09-10 18:42:19.986 [INFO] [test_logger_node.test]: Hello from another Logger!


#### By default, the Node name is used as prefix, which can be overwritten (either name or prefix is required).

In [17]:
node.second_logger.set_settings({'prefix': ""})

In [18]:
node.second_logger.info("Hello without a prefix!")
None

2025-09-10 18:42:20.404 [INFO] [test]: Hello without a prefix!


#### The severity and name of a Logger can be linked to another Logger.

In [19]:
node.logger_one = Logger(node, settings={'severity': 20, 'name': "one"})

In [20]:
node.logger_one.info("Test")
None

2025-09-10 18:42:20.202 [INFO] [test_logger_node.one]: Test


In [21]:
node.logger_two = Logger(node, settings={'severity': "logger_one", 'prefix': "logger_one", 'name': "two"})

In [22]:
node.logger_two.info("Test")
None

2025-09-10 18:42:20.384 [INFO] [test_logger_node.one.two]: Test


In [23]:
print("Name of Logger 2:", node.logger_two.get_name())
print("Severity of Logger 2:", node.logger_two.get_severity())

node.logger_one.set_settings({'severity': 10, 'name': "one_edit"})
print()

print("Name of Logger 2 after setting Logger 1:", node.logger_two.get_name())
print("Severity of Logger 2 after setting Logger 1:", node.logger_two.get_severity())

Name of Logger 2: test_logger_node.one.two
Severity of Logger 2: 20

Name of Logger 2 after setting Logger 1: test_logger_node.one_edit.two
Severity of Logger 2 after setting Logger 1: 10


In [24]:
node.logger_two.debug("Test")
None

2025-09-10 18:42:20.640 [DEBUG] [test_logger_node.one_edit.two]: Test


## Stop Node

In [25]:
stop_node(node, executor, context, thread)

> Stopped node 'MyNode'
